In [ ]:
%%bash
# preamble script to check and install AMUSE components if necessary

# required packages for this tutorial:
PACKAGES="mpi4py amuse-framework amuse-bhtree"
# skip in case a full development install is present
pip show amuse-devel && exit 0
for package in ${PACKAGES} 
do
  pip show ${package} || pip install ${package}
done

In [ ]:
# the following fixes are highly recommended

#allow oversubscription for openMPI
import os
os.environ["OMPI_MCA_rmaps_base_oversubscribe"]="true"

# use lower cpu resources for idle codes
from amuse.support import options
options.GlobalOptions.instance().override_value_for_option("polling_interval_in_milliseconds", 10)


AMUSE uses numpy for numerical data and matplotlib to plot, we start by importing them:

In [1]:
%matplotlib inline
from matplotlib import pyplot
import numpy

The AMUSE framework is divided over several python packages and modules. All these have one common root module: _amuse_. This root defines the namespace of all sub-packages and modules. You can load the amuse module with:

In [2]:
import amuse

However the _amuse_ module is mostly empty, as you can see when you try 'Tab' completion on the amuse module or run dir(amuse)

In [3]:
dir(amuse)

['NoConfig',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'config',
 'numpy',
 'numpy_fix',
 'support']

The main body of the AMUSE framework is divided over several packages, each subdivided into more packages and modules.

### units ###

A package to work with quantities and units. All calculations in AMUSE are done with quantities having units. These quantities and their units are implemented as python classes and can be used almost everywere you would normaly use a number (or a `numpy` array). In the next tutorial we will come back to the units, for now we will show a simple example 

In [4]:
from amuse.units import units
from amuse.units import constants

In [5]:
constants.G * (5.972e24 | units.kg) /  (6378.1 | units.km )**2

quantity<9.79809335610421 m * s**-2>

### datamodel ###

All astrophysical bodies (stars, clouds, planets etc) are modelled with sets of particles or on grids. These sets and grids are defined in the _datamodel_ package. You will see these used everywhere in amuse and in several tutorials we will cover them in more detail. 

In [6]:
from amuse.datamodel import Particles
solar_system_planets = Particles(7)
solar_system_planets.mass = [641.85, 4868.5, 5973.6, 102430, 86832, 568460, 1898600] | (1e21 * units.kg)
print(solar_system_planets)

                 key         mass
                   -   1e+21 * kg
====================  ===========
16568001987451464039    6.419e+02
 7207579071768592374    4.868e+03
 2593871659432541062    5.974e+03
12625977258096592293    1.024e+05
17547242889775356555    8.683e+04
 3006279325155648284    5.685e+05
 3570298971428347869    1.899e+06
====================  ===========


### io ###

*(this example depends on the datamodel example earlier on this page, if you get a NameError please rerun that cell)*

Saving and loading models is an important feature in AMUSE. AMUSE defines it's own format and supports a number of common file formats often used in astro-physical codes. All saving all loading is supported through two functions: 

In [7]:
from amuse import io
io.write_set_to_file(solar_system_planets, "solar.hdf5")
print(io.read_set_from_file("solar.hdf5"))

                 key         mass
                   -   1e+21 * kg
====================  ===========
 8306438798998442450    6.419e+02
 8953103678612694330    4.868e+03
 5729276014273372650    5.974e+03
 3011160290541337866    1.024e+05
10218094749956874519    8.683e+04
 9955606421551439791    5.685e+05
 2508219077035151455    1.899e+06
====================  ===========


All data is written to the current working directory, which is:

In [8]:
import os
os.getcwd()

'/Users/srieder/Code/amuse/doc/interactive_tutorial'

### community ###

This packages contains all the communtity codes. All software integrated into AMUSE, (N-body, stellar evolution, hydrodynamics, radiative transfer and other codes) is called a _community_ code in AMUSE. Each community code is defined in a separate sub-package and every sub-package contains at least one module called `interface`.
You can load a community code with (note the capitalisation):
```python
from amuse.community.<codename>.interface import <Codeclass>
```
In later tutorials we will learn more about the codes and how to interact with each.

In [9]:
from amuse.community.bhtree.interface import Bhtree
print(Bhtree)

<class 'amuse.community.bhtree.interface.BHTree'>


### ic ###
The Initial Conditions _ic_ package provides a small set of often used algorithms to create an initial particle set or grid. The current set of supported algorithms is small but provides good examples for further extension. 

In [10]:
from amuse.ic import plummer
particles = plummer.new_plummer_model(25)
print(particles.mass.sum())

1.0 mass


### couple ###

Eventually you will want to connect different codes to simulate a coupled problem. As AMUSE is written in Python, you can easily write your own coupling strategy but in most cases you do not need to do this. This package contains several predefined coupling strategies, two of which we will cover in a later tutorial.

In [11]:
from amuse.couple import bridge

### ext ###
During development of AMUSE we have created several general purpose algorithms that do not fall in any of the above categories. All these are implemented in the modules of the _ext_ package. These algorithms are very diverse and deal with model conversions (from a grid model to particle sets), post-processing (graphing, creating blender models) to handling of special conditions in a model  (sinks and sources). Some of these algorithms will evetually move to the _ic_ or _couple_ modules. We will encounter some _ext_ modules in these tutorials and you will see more use in the examples.


In [12]:
from amuse.ext import grid_to_sph

### all in one ###

Most amuse code can be loaded with importing one module, the _lab_ module. AMUSE contains a lot of sub-packages and modules, it is hard too learn and remember all these. We provide one module that imports most code from the other modules, as an easy interface to the AMUSE framework.

In [13]:
from amuse.lab import *
print(new_plummer_model(10))

                 key         mass       radius           vx           vy           vz            x            y            z
                   -         mass       length  length / time  length / time  length / time       length       length       length
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========
 6891505399092943826    1.000e-01    0.000e+00   -1.687e-01   -2.299e-01    6.470e-01    9.172e-02   -2.919e-01   -2.517e-01
 9236984259963745259    1.000e-01    0.000e+00    1.109e-01   -3.582e-01    2.355e-01   -1.173e+00    1.264e+00    3.269e-01
12473597854400827686    1.000e-01    0.000e+00   -3.864e-02    3.738e-01    6.397e-01    2.299e-01   -3.756e-01   -2.306e-01
 8748970469354843124    1.000e-01    0.000e+00   -3.417e-01   -9.798e-01    1.653e-01   -4.780e-01   -3.624e-01    3.556e-01
10514419277732552049    1.000e-01    0.000e+00   -1.342e-01    4.418e-01   -1.722e-02    6.351e-01   -5.376e-01    1.25